# TME 2 

## 1. Introduction

In [1]:
import pandas as pd
import itertools as it
import numpy as np
#from ete3 import Tree
import math

In [2]:
M1 = [[0,8,7,12], [8,0,9,14], [7,9,0,11], [12,14,11,0]]
M2 = [[0,2,3,8,14,18],[2,0,3,8,14,18],
      [3,3,0,8,14,18],[8,8,8,0,14,18],
      [14,14,14,14,0,18],[18,18,18,18,18,0]]
#UPGMA
M3 = [[0,19,27,8,33,18,13],[19,0,31,18,36,1,13],
          [27,31,0,26,41,32,29],[8,18,26,0,31,17,14],
          [33,36,41,31,0,35,28],[18,1,32,17,35,0,12],
          [13,13,29,14,28,12,0]]
#Neighbor Joining
M4 = [[0,2,4,6,6,8],[2,0,4,6,6,8],
          [4,4,0,6,6,8],[6,6,6,0,4,8],
          [6,6,6,4,0,8],[8,8,8,8,8,0]]

M1D = pd.DataFrame(M1)
M2D = pd.DataFrame(M2)
UPGMAD = pd.DataFrame(M3, index= list("ABCDEFG"), columns = list("ABCDEFG")) 
NJD = pd.DataFrame(M4)

M1D

,0,1,2,3
0,0,8,7,12
1,8,0,9,14
2,7,9,0,11
3,12,14,11,0


In [5]:
def convert(M):
    M_array = np.array(M).astype(float)
    il1 = np.triu_indices(len(M_array))
    M_array[il1] = np.nan
    MD = pd.DataFrame(M_array, index = M.index, columns = M.columns)
    return MD

convert(M1D)

,0,1,2,3
0,NaN,NaN,NaN,NaN
1,8.0,NaN,NaN,NaN
2,7.0,9.0,NaN,NaN
3,12.0,14.0,11.0,NaN


###### Question 1

##### Matrice additive

Une est matrice additive si et seulement si la condition des 4 points est vraie pour tous les quartets i,j,k et l où 1<=i,j,k,l<=n).
La condition des 4 points est remplie quand Dij + Dkl <= max(Dik+Djl,Dil+Djk).

In [6]:
def estadditive(M):
    comb=it.combinations(range(len(M)),4)
    for groupe in comb:
        i,j,k,l=groupe
        if not (M.at[i,j]+M.at[k,l]<=max(M.at[i,k]+M.at[j,l],M.at[i,l]+M.at[j,k])):
            return False
    return True

In [7]:
print(M1D,"\n",estadditive(M1D))

    0   1   2   3
0   0   8   7  12
1   8   0   9  14
2   7   9   0  11
3  12  14  11   0 
 True


##### Ultramétricité

Une matrice est dite ultramétrique quand, pour tout choix de 3 points i,j,k au moins deux distances sont égales : Dij et Djk.

On a alors mathématiquement : Dij = Djk >Dik

Une matrice ultramétrique est forcément additive.

In [8]:
def estultra(M):
    comb=it.combinations(range(len(M)),3)
    for groupe in comb:
        i,j,k=groupe
        if not(M.at[i,k] <= max(M.at[i,j],M.at[j,k])):
            return False
    return True

In [9]:
print(M2D,"\n", estultra(M2D))

    0   1   2   3   4   5
0   0   2   3   8  14  18
1   2   0   3   8  14  18
2   3   3   0   8  14  18
3   8   8   8   0  14  18
4  14  14  14  14   0  18
5  18  18  18  18  18   0 
 True


###### Question 2

In [10]:
#Calcul de la somme des distances d'un cluster avec les autres
def cluster(M,i):            #i = l'espèce en question
    return sum(M.iloc[i])

In [11]:
#Calcul de toutes les sommes de distances entre les différentes clusters 
def groupe(M):
    L=[]
    for k in range(len(M)):
        L.append(cluster(M,k))
    return L


In [12]:
def group_clust_df(M):
    return M.sum()

In [13]:
cluster(M1D,1)
groupe(M1D)

group_clust_df(M1D)

0    27
1    31
2    27
3    37
dtype: int64

## 2 . UPGMA

###### Question 1 

Le format Newick est un format de fichier utilisé en phylogénie pour décrire les relations phylogénétiques entre différents clusters.
Par exemple si on a un ancêtre commun entre A et B puis entre AB et C on écrira sous ce format :

(C : longueur de la branche (A : longueur, B : longueur) : longueur restante de la branche)

L'extension de ce format est .nwk

In [12]:
UPGMAD

,A,B,C,D,E,F,G
A,0,19,27,8,33,18,13
B,19,0,31,18,36,1,13
C,27,31,0,26,41,32,29
D,8,18,26,0,31,17,14
E,33,36,41,31,0,35,28
F,18,1,32,17,35,0,12
G,13,13,29,14,28,12,0


In [13]:
"""""
# Input : nwk_dico = dictionnaire contenant les codes nwk
          dmin_name = nom du nouveau noeud, 
          d_min_value = valeur de la distance des deux espèces du noeud

# Fonctionnement (Application de l'algorithme  Neighbor Joining): 
    1. Attribution d'un code nwk aux espece composite du noeud
    2. Enregistrement du noeud et du code nwk correspondant
    3. Supression des valeurs des noeuds/espèces ayant dèjà fusionnées

# Output : Dico contenant le code nwk mis à jour

"""""
def refresh_nwk(dico, dmin_name, d_min, d_min_value):
    
    nwk = dico.copy()
    
    for i in d_min:
        if not str(i) in nwk:
            nwk[str(i)] = str(i) + ":" + str(d_min_value/2)
    
    if(cardinal(d_min[0]) == 1 and cardinal(d_min[1]) == 1):
        nwk[dmin_name] = "(" + nwk[str(d_min[0])] + "," + nwk[str(d_min[1])] + ")" 
    elif(cardinal(d_min[0]) > 1 and cardinal(d_min[1]) == 1):
        nwk[dmin_name] = "(" + nwk[str(d_min[0])] + ":" + str(d_min_value/2 - nwk[''.join([str(d_min[0]),"_value"])]) + "," + nwk[str(d_min[1])] + ")" 
    elif(cardinal(d_min[0]) == 1 and cardinal(d_min[1]) > 1):
        nwk[dmin_name] = "(" + nwk[str(d_min[0])] + "," + nwk[str(d_min[1])] + ":" + str(d_min_value/2 - nwk[''.join([str(d_min[1]),"_value"])]) + ")" 
    else:
        nwk[dmin_name] = "(" + nwk[str(d_min[0])] + ":" + str(d_min_value/2 - nwk[''.join([str(d_min[0]),"_value"])]) + "," + nwk[str(d_min[1])] + ":" + str(d_min_value/2 - nwk[''.join([str(d_min[1]),"_value"])]) + ")" 
    
    nwk[''.join([dmin_name,"_value"])] = d_min_value/2
    
    for i in d_min:
        nwk.pop(str(i))
        if cardinal(i) > 1:
                nwk.pop(''.join([str(i),"_value"]))
                
    return nwk   
 
"""""
# Input : data = nom du noeud

# Fonctionnement : Calcul du cardinal en fonction du nombre de chiffre/lettre de la variable à analyser
                   Contrainte : Une espèce ne peut être défini qu'avec UN chiffre ou UNE lettre
# Output : valeur du cardinal (int)

"""""
# Calcul du cardinal (nom de la feuille doit être un character ou un chiffre)
def cardinal(data):
    data_str = str(data)
    cardinal = 0
    for s in data_str:
        if s.isnumeric() or s.isalpha():
            cardinal += 1
    return cardinal

"""""
# Input : M = Dataframe à analyser

# Fonctionnement (Application de l'algorithme UPGMA): 
    1. Recherche de la distance minamale entre les noeuds/espèces
    3. Création du noeud correspondant
    4. Mise à jour de code nwk à partir de ce nouveau noeud
    5. Mise à jour de la matrice avec calcul des distances du nouveau noeud

# Output : Code nwk correspondant au résultat de l'algorithme UPGMA
            + affichage de chaque itérations + affichage de l'arbre

"""""
def UPGMA(M):
    
    matrix_UPGMA = M.copy()
    nwk_dico = {}
    count = 0
    
    #Boucle jusqu'à que la matrice ne contienne plus qu'un noeud
    while(matrix_UPGMA.shape != (1,1)):
        count += 1
        
        #Recherche de la distance minimale
        d_min = convert(matrix_UPGMA).stack().idxmin()
        d_min_name = ''.join(str(x) for x in d_min)
        d_min_value = matrix_UPGMA.at[d_min[0], d_min[1]]
        
        #Enregistrement du nouveau noeud au format newick et sauvegarde des valeurs de distances
        nwk_dico = refresh_nwk(nwk_dico, d_min_name, d_min, d_min_value)
        
        #Création d'une nouvelle matrix avec le nouveau noeud
        new_row = np.zeros(len(matrix_UPGMA))
        new_row[:] = np.nan 
        
        new_matrix_UPGMA = matrix_UPGMA.copy()

        new_matrix_UPGMA.loc[d_min_name] = new_row
        new_matrix_UPGMA.insert(loc=0, column= d_min_name, value = np.NaN)
        new_matrix_UPGMA = new_matrix_UPGMA.reindex(new_matrix_UPGMA.columns[:])
        new_matrix_UPGMA.drop(list(d_min), axis=0, inplace=True)
        new_matrix_UPGMA.drop(list(d_min), axis=1, inplace=True)
            
        #Attribution de distances au nouveau noeud dans la matrice
        for i in new_matrix_UPGMA.columns[1:]:
            new_matrix_UPGMA.at[d_min_name,i] = new_matrix_UPGMA.at[i,d_min_name] = (cardinal(d_min[0])*matrix_UPGMA.at[d_min[0],i]+cardinal(d_min[1])*matrix_UPGMA.at[d_min[1],i])/(cardinal(d_min[0])+cardinal(d_min[1]))
        
        #Mise à jour de la matrice
        matrix_UPGMA = new_matrix_UPGMA.copy()
        
        #Affichage du résultat
        print("~~~~~~~~~~~itération n°", count, "~~~~~~~~~~~")
        print(convert(matrix_UPGMA))
        
    #Affichage du résultat final
    print("\n Code nwk :", nwk_dico[d_min_name]+";")
    print(Tree(str(list(nwk_dico.values())[0]+";")))
    
    return nwk_dico[d_min_name]
        
        
    
    

In [14]:
UPGMA(UPGMAD)

~~~~~~~~~~~itération n° 1 ~~~~~~~~~~~
      FB     A     C     D     E   G
FB   NaN   NaN   NaN   NaN   NaN NaN
A   18.5   NaN   NaN   NaN   NaN NaN
C   31.5  27.0   NaN   NaN   NaN NaN
D   17.5   8.0  26.0   NaN   NaN NaN
E   35.5  33.0  41.0  31.0   NaN NaN
G   12.5  13.0  29.0  14.0  28.0 NaN
~~~~~~~~~~~itération n° 2 ~~~~~~~~~~~
      DA    FB     C     E   G
DA   NaN   NaN   NaN   NaN NaN
FB  18.0   NaN   NaN   NaN NaN
C   26.5  31.5   NaN   NaN NaN
E   32.0  35.5  41.0   NaN NaN
G   13.5  12.5  29.0  28.0 NaN
~~~~~~~~~~~itération n° 3 ~~~~~~~~~~~
           GFB    DA     C   E
GFB        NaN   NaN   NaN NaN
DA   16.500000   NaN   NaN NaN
C    30.666667  26.5   NaN NaN
E    33.000000  32.0  41.0 NaN
~~~~~~~~~~~itération n° 4 ~~~~~~~~~~~
       DAGFB     C   E
DAGFB    NaN   NaN NaN
C       29.0   NaN NaN
E       32.6  41.0 NaN
~~~~~~~~~~~itération n° 5 ~~~~~~~~~~~
        CDAGFB   E
CDAGFB     NaN NaN
E         34.0 NaN
~~~~~~~~~~~itération n° 6 ~~~~~~~~~~~
         ECDAGFB
ECDAGF

'(E:17.0,(C:14.5,((D:4.0,A:4.0):4.25,(G:6.25,(F:0.5,B:0.5):5.75):2.0):6.25):2.5)'

## 3 . NJ

#### 1. Etant données une matrice de distance et une espèce (cluster i), créer une fonction retournant la valeur de u_i .

In [ ]:
"""""
# Input : matrix_dist = matrice de distance,
          espece_name = nom de l'espèce

# Fonction retournant la valeur de u_i

# Output : valeur de u_i

"""""
def calcul_ui(matrix_dist, espece_name):
    
    return matrix_dist.loc[:,espece_name].sum() / (len(matrix_dist.columns)-2)

#### 2. Etant donnés une matrice de distance et un couple d'espèces (clusters i et j), créer une fonction permettant de calculer Q_ij .


In [ ]:
"""""
# Input : matrix_dist = matrice de distance,
          espece_name_i = nom de l'espèce i,
          espece_name_j = nom de l'espèce j,

# Fonction retournant la valeur de Q_ij

# Output : valeur de Q_ij

"""""
def calcul_Qij(matrix_dist, espece_name_i, espece_name_j):
    
    return matrix_dist.at[espece_name_i,espece_name_i] - calcul_ui(matrix_dist, espece_name_i) - calcul_ui(matrix_dist, espece_name_j)


#### 3. En utilisant ces fonctions et en se basant sur les diérentes étapes présentées, proposer une fonction permettant de retourner le résultat de l'algorithme UPGMA sous le format Newick.

In [18]:
"""""
# Input : nwk_dico = dictionnaire contenant les codes nwk, 
          u_dico = dictionnaire des distances des espèces,
          dmin_name = nom du nouveau noeud,
          d_min_value = valeur de la distance des deux espèces du noeud
          final = variable modifiant le traitement des données de la fonction

# Fonctionnement (Application de l'algorithme  Neighbor Joining): 
    1. Attribution d'un code nwk aux espece composite du noeud
    2. Enregistrement du noeud et du code nwk correspondant à partir des codes nwk déjà établies
    3. Supression des valeurs des noeuds/espèces ayant dèjà fusionnées

# Output : Dico contenant le code nwk mis à jour

"""""
def refresh_nwk_NJ(nwk_dico, u_dico, dmin_name, d_min, d_min_value, final):
    
    nwk = nwk_dico.copy()
    
    for i in d_min:
        if not str(i) in nwk:
            nwk[str(i)] = str(i)
    #Si il reste plus de 2 noeuds
    if final == False:
        nwk[dmin_name] = "(" + nwk[str(d_min[0])] + ":" + str(d_min_value/2 + (u_dico[d_min[0]] - u_dico[d_min[1]])/2) + "," + nwk[str(d_min[1])] + ":" + str(d_min_value/2 + (u_dico[d_min[0]] - u_dico[d_min[1]])/2) + ")" 
    #Seulement 2 noeuds restant
    else:
        nwk[dmin_name] = "(" + nwk[str(d_min[0])] + ":" + str(d_min_value) + "," + nwk[str(d_min[1])] + ":" + str(d_min_value) + ")" 
    
    for i in d_min:
        nwk.pop(str(i))
                
    return nwk
"""""
# Input : nwk_dico = dictionnaire contenant les codes nwk, 
          u_dico = dictionnaire des distances des espèces,
          dmin_name = nom du nouveau noeud,
          d_min_value = valeur de la distance des deux espèces du noeud
          final = variable modifiant le traitement des données de la fonction

# Fonctionnement (Application de l'algorithme  Neighbor Joining): 
    1. Attribution d'un code nwk aux espece composite du noeud
    2. Enregistrement du noeud et du code nwk correspondant à partir des codes nwk déjà établies
    3. Supression des valeurs des noeuds/espèces ayant dèjà fusionnées

# Output : Dico contenant le code nwk mis à jour

"""""
def refresh_nwk_NJ(nwk_dico, u_dico, dmin_name, d_min, d_min_value, final):
    
    nwk = nwk_dico.copy()
    
    for i in d_min:
        if not str(i) in nwk:
            nwk[str(i)] = str(i)
    #Si il reste plus de 2 noeuds
    if final == False:
        nwk[dmin_name] = "(" + nwk[str(d_min[0])] + ":" + str(d_min_value/2 + (u_dico[d_min[0]] - u_dico[d_min[1]])/2) + "," + nwk[str(d_min[1])] + ":" + str(d_min_value/2 + (u_dico[d_min[0]] - u_dico[d_min[1]])/2) + ")" 
    #Seulement 2 noeuds restant
    else:
        nwk[dmin_name] = "(" + nwk[str(d_min[0])] + ":" + str(d_min_value) + "," + nwk[str(d_min[1])] + ":" + str(d_min_value) + ")" 
    
    for i in d_min:
        nwk.pop(str(i))
                
    return nwk

"""""
# Input : nwk_dico = dictionnaire contenant les codes nwk, 
          u_dico = dictionnaire des distances des espèces,
          dmin_name = nom du nouveau noeud,
          d_min_value = valeur de la distance des deux espèces du noeud
          final = variable modifiant le traitement des données de la fonction

# Fonctionnement (Application de l'algorithme  Neighbor Joining): 
    1. Attribution d'un code nwk aux espece composite du noeud
    2. Enregistrement du noeud et du code nwk correspondant à partir des codes nwk déjà établies
    3. Supression des valeurs des noeuds/espèces ayant dèjà fusionnées

# Output : Dico contenant le code nwk mis à jour

"""""
def refresh_nwk_NJ(nwk_dico, u_dico, dmin_name, d_min, d_min_value, final):
    
    nwk = nwk_dico.copy()
    
    for i in d_min:
        if not str(i) in nwk:
            nwk[str(i)] = str(i)
    #Si il reste plus de 2 noeuds
    if final == False:
        nwk[dmin_name] = "(" + nwk[str(d_min[0])] + ":" + str(d_min_value/2 + (u_dico[d_min[0]] - u_dico[d_min[1]])/2) + "," + nwk[str(d_min[1])] + ":" + str(d_min_value/2 + (u_dico[d_min[0]] - u_dico[d_min[1]])/2) + ")" 
    #Seulement 2 noeuds restant
    else:
        nwk[dmin_name] = "(" + nwk[str(d_min[0])] + ":" + str(d_min_value) + "," + nwk[str(d_min[1])] + ":" + str(d_min_value) + ")" 
    
    for i in d_min:
        nwk.pop(str(i))
                
    return nwk

"""""

# Input : M = Dataframe à analyser

# Fonctionnement (Application de l'algorithme  Neighbor Joining): 
    1. Pour chaque espèce calcul de sa distance par rapport au reste de l'arbre
    2. Calcul de la paire de distance minimale
    3. Création du noeud correspondant
    4. Mise à jour de code nwk à partir de ce nouveau noeud
    5. Mise à jour de la matrice avec calcul des distance du nouveau noeud

# Output : Code nwk correspondant au résultat de l'algorithme Neighbor Joining 
            + affichage de chaque itérations + affichage de l'arbre

"""""


def NJ(M):
    
    
    #Initialisation
    matrix_NJ = M.copy()
    nwk_dico = {}
    u_dico = {}
    count = 0
    d_min = []
    
    while(matrix_NJ.shape != (2,2)):
        
        #Initialisation
        count += 1
        u_min_value = math.inf
        
        #Calcul des distances de tous les u_i
        for i in matrix_NJ.columns:
            u_dico[i] = calcul_ui(matrix_NJ, i)
        
        #Recherche de i et j tel que D_ij - u_i - u_j est minimale
        for j in it.combinations(list(matrix_NJ.columns),2):
            # Possibilité d'écrire calcul_Qij(matrix_NJ, i, j) à la place de 
            #  matrix_NJ.at[j[0],j[1]] - u_dico[j[0]] - u_dico[j[1]]
            #  mais cela prends plus de temps car fait tourner la fonction calcul_ui plus de fois
            if (matrix_NJ.at[j[0],j[1]] - u_dico[j[0]] - u_dico[j[1]]) < u_min_value :
                u_min_value = matrix_NJ.at[j[0],j[1]] - u_dico[j[0]] - u_dico[j[1]]
                print(True)
                d_min = j
        
        #Enregistrement de la distance minimale
        d_min_name = ''.join(str(x) for x in d_min)
        d_min_value = matrix_NJ.at[d_min[0], d_min[1]] 
        
        #Mise à jour du code nwk
        nwk_dico = refresh_nwk_NJ(nwk_dico, u_dico, d_min_name, d_min, d_min_value, False)
        
        #Création d'une nouvelle matrix avec le nouveau noeud
        new_row = np.zeros(len(matrix_NJ))
        new_row[:] = np.nan 
        
        new_matrix_NJ = matrix_NJ.copy()

        new_matrix_NJ.loc[d_min_name] = new_row
        new_matrix_NJ.insert(loc=0, column= d_min_name, value = np.NaN)
        new_matrix_NJ = new_matrix_NJ.reindex(new_matrix_NJ.columns[:])
        new_matrix_NJ.drop(list(d_min), axis=0, inplace=True)
        new_matrix_NJ.drop(list(d_min), axis=1, inplace=True)
            
        #Attribution de distances au nouveau noeud dans la matrice
        for i in new_matrix_NJ.columns[1:]:
            new_matrix_NJ.at[d_min_name,i] = new_matrix_NJ.at[i,d_min_name] = (matrix_NJ.at[d_min[0], i] + matrix_NJ.at[d_min[1], i] - d_min_value)/2
        
        #Mise à jour de la matrice
        matrix_NJ = new_matrix_NJ.copy()
        print("~~~~~~~~~~~itération n°", count, "~~~~~~~~~~~")
        print(convert(matrix_NJ))
    
    #Quand il ne reste plus que deux noeuds
    d_min = matrix_NJ.columns
    d_min_name = ''.join(str(x) for x in d_min)
    d_min_value = matrix_NJ.at[d_min[0], d_min[1]] 
    nwk_dico = refresh_nwk_NJ(nwk_dico, u_dico, d_min_name, d_min, d_min_value, True)
         
    print("\n Code nwk :", nwk_dico[d_min_name]+";")
    print(Tree(str(list(nwk_dico.values())[0]+";")))
    
    return nwk_dico[d_min_name]

In [16]:
NJ(NJD)

True
~~~~~~~~~~~itération n° 1 ~~~~~~~~~~~
     01    2    3    4   5
01  NaN  NaN  NaN  NaN NaN
2   3.0  NaN  NaN  NaN NaN
3   5.0  6.0  NaN  NaN NaN
4   5.0  6.0  4.0  NaN NaN
5   7.0  8.0  8.0  8.0 NaN
True
~~~~~~~~~~~itération n° 2 ~~~~~~~~~~~
     012    3    4   5
012  NaN  NaN  NaN NaN
3    4.0  NaN  NaN NaN
4    4.0  4.0  NaN NaN
5    6.0  8.0  8.0 NaN
True
True
~~~~~~~~~~~itération n° 3 ~~~~~~~~~~~
      0125    3   4
0125   NaN  NaN NaN
3      3.0  NaN NaN
4      3.0  4.0 NaN
True
~~~~~~~~~~~itération n° 4 ~~~~~~~~~~~
       01253   4
01253    NaN NaN
4        2.0 NaN

 Code nwk : (((((0:1.0,1:1.0):1.0,2:1.0):1.0,5:1.0):1.0,3:1.0):2.0,4:2.0);

               /-0
            /-|
         /-|   \-1
        |  |
      /-|   \-2
     |  |
   /-|   \-5
  |  |
--|   \-3
  |
   \-4


'(((((0:1.0,1:1.0):1.0,2:1.0):1.0,5:1.0):1.0,3:1.0):2.0,4:2.0)'